# Take data from normPosts.csv and create dataForGraph.json

In [1]:
#normPosts.json comes from normPosts.ipynb

import pandas as pd
df = pd.read_json('normPosts.json')
print(f'read {len(df)} posts')
df = df.sort_values('timeInt')
df.iloc[:2,:]

read 76238 posts


,ID,author,body,body_html,category,comments,date_created,isoformat,members_only,timeInt,title
0,0.000,"{'username': 'jonschull', 'avatar': 'https://w...",Welcome to the companion site to this map[http...,<p>Welcome to the companion site to this map<a...,"Legal, Documents, $$","[{'date_created': '2013-07-10T20:36:01+00:00',...",2013-07-10T20:01:06+00:00,2013-07-10T14:01:06-06:00,1.0,1373486466,Welcome to the companion site to this map...
1,0.001,"{'username': 'jonschull', 'avatar': 'https://w...",short link: http://goo.gl/1a4pb,"<p>short link: <a href=""http://goo.gl/1a4pb"">...",None,None,2013-07-10T20:36:01+00:00,2013-07-10T14:36:01-06:00,NaN,1373488561,None


### we want to assign userNameNumbers in order of appearance

In [2]:
#give each record a username
usernames = [a['username'] for a in df['author']]
df['username'] = usernames

print(f'len df {len(df)}')
print(f'len usernames {len(usernames)}')

#assign uniqueUserNameNumbers

#first collect unique Names
uniqueUserNames = [] #these should preserve order in which usernames appear on the scene
for username in usernames:
    if username not in uniqueUserNames:
        uniqueUserNames.append(username)

#associate numbers to unique names
userNameNumbers = dict()
for i, uUN in enumerate(uniqueUserNames):
    userNameNumbers[uUN] = i

#for the Graph
ys = [userNameNumbers[username] for username in usernames]
colors = [ y % 12 for y in ys]


df['ts'] = df['isoformat'] #just so it's here
df['ys'] = ys
df['colors']  = colors


len df 76238
len usernames 76238


## create 2 jsons with just what we need

In [3]:
#### beware: csvs sometimes choke on HTML

In [4]:
bodies = []
for tup in df.itertuples():
    entry = dict(
        ID=tup.ID, 
        username=tup.username,
        isoformat = tup.isoformat,
        body=tup.body, 
        body_html=tup.body_html )
    bodies.append(entry)
bodiesDF = pd.DataFrame(bodies)
#bodiesDF.to_csv('dataForHoverPanel.csv', index=False)
#print('created dataForHoverPanel.csv')

#bodiesDF.to_json('dataForHoverPanel.json')
#print('created dataForHoverPanel.json')

In [5]:
df['texts'] = [post['username'] + ' ' +  post['ts'].replace('T',' ') for post in df.to_dict('records')]
#df = df.drop(['username', 'author', 'body', 'body_html', 'date_created',],axis=1)
df = df.drop(['author', 'date_created',],axis=1)
df = df.fillna('null')
df.to_json('dataForGraph.json')

print('created dataForGraph.json')

created dataForGraph.json


In [6]:
MAKINGFIGURE= False
if MAKINGFIGURE:
    from IPython.display import display
    
    newdf=pd.read_json('dataForGraph.json')
    
    import plotly.graph_objs as go
    fig = go.Figure(
        dict(
            data = [
                go.Scattergl(
                    x = newdf['ts'],
                    y=  newdf['ys'],
                    mode='markers',
                    marker=dict(
                        size=4,
                        symbol='square',
                        color= newdf['colors'],
                        colorscale='Jet'
                    ),
                    name='JonTest',
                    hovertext =  newdf['texts'],
                    hoverinfo = 'text', #could be changed for Date and Name
                    #customdata= newdf['customs']
                )
            ],

            layout = go.Layout(
                hovermode='closest',
                title="People and Posts over time, colored by Person",
                xaxis_title="Time",
                yaxis_title="People",
                height=1000,
                width=1500
            )
        ) 
    )
    
    display(fig)